# <span style="color: #FF69B4;">Christhian O. Rosado </span><br >
<img src="http://i.stack.imgur.com/XIM7f.png" alt="Smiley face" height="100" width="100" style="float: left; margin: 5px 15px 10px 10px;">

### Assignment 1: Compare Tests for Goodness of fit (on real data)

Test whether a gaussian model N($\mu$, $\sigma$) for the age distribution of citibike drivers is a sensible model, or if you can find a better fit with another distribution.

Use 2 tests: KS, AD, KL, chisq (even though we have not talked about it in detail yet) to do this.

Test at the Normal and a least one other distributions (e.g. Poisson, or Binomial, or Chisq, Lognormal.......)

No skeleton: you are on your own!

Extra credit: Divide your sample geographically: by Borrow + split Manhattan in an Uptown and a Downtown sample (use your discretion to do so, but ZIP code is a good idea) and see if you notice any differences in how the age distribution can be modeled.

### GRADING:

Your notebook must:

state the $H_0$ correcty <br >
generate the distributions correctly. <br >
use the test correctly <br >
make conclusions about the rejection of the Null <br >
each plot must have a caption which describes the plot in terms of Central Limit Theorem <br >

<br >
<img src="https://archive.today/HNSMb/64108f29f5f0307ea35b6cb0a6fa31886260cc34.png" alt="Smiley face" height="" width="100" style="float: left; margin: 5px 15px 10px 10px;">
<img src="https://archive.today/HNSMb/64108f29f5f0307ea35b6cb0a6fa31886260cc34.png" alt="Smiley face" height="" width="100" style="float: left; margin: 5px 15px 10px 10px;">
<img src="https://archive.today/HNSMb/64108f29f5f0307ea35b6cb0a6fa31886260cc34.png" alt="Smiley face" height="" width="100" style="float: left; margin: 5px 15px 10px 10px;">
<img src="https://archive.today/HNSMb/64108f29f5f0307ea35b6cb0a6fa31886260cc34.png" alt="Smiley face" height="" width="100" style="float: left; margin: 5px 15px 10px 10px;">
<img src="https://archive.today/HNSMb/64108f29f5f0307ea35b6cb0a6fa31886260cc34.png" alt="Smiley face" height="" width="100" style="float: left; margin: 5px 15px 10px 10px;"><img src="https://archive.today/HNSMb/64108f29f5f0307ea35b6cb0a6fa31886260cc34.png" alt="Smiley face" height="" width="100" style="float: left; margin: 5px 15px 10px 10px;">
<img src="https://archive.today/HNSMb/64108f29f5f0307ea35b6cb0a6fa31886260cc34.png" alt="Smiley face" height="" width="100" style="float: left; margin: 5px 15px 10px 10px;">

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy

%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [9]:
### A function to download the data: simple bash commands can be run by preceding them with !
### but more complex command lines with variable arguments need to be build as strng and run with os.system

def getCitiBikeCSV(datestring):
    print ("Downloading", datestring)
    ### First I will check that it is not already there
                        # str(os.getenv("PUIDATA")) + "/"
    if not os.path.isfile("PUIDATA/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
                                                                        # str(os.getenv("PUIDATA"))
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + "PUIDATA"):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
                                # str(os.getenv("PUIDATA")) + "/"
            if not os.path.isfile("PUIDATA/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
            ###  To move it I use the os.system() functions to run bash commands with arguments
                                                                     # str(os.getenv("PUIDATA"))
            os.system("mv " + datestring + "-citibike-tripdata.zip " + "PUIDATA")

            os.system("sudo apt-get install unzip")
            ### unzip the csv 
            ### unzip file.zip -d destination_folder
                               # str(os.getenv("PUIDATA")) + "/"
                               # + datestring + "-citibike-tripdata.zip" + " -d /PUIDATA"
                               # + "/PUIDATA/" + datestring + "-citibike-tripdata.zip"
            os.system("unzip " + "PUIDATA/" + datestring + "-citibike-tripdata.zip")
            
               
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
                                                                         # str(os.getenv("PUIDATA"))
                os.system("mv " + datestring + "-citibike-tripdata.csv " + "/PUIDATA")
    ### One final check:
                        # str(os.getenv("PUIDATA")) + "/"
    if not os.path.isfile("PUIDATA/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")


In [12]:
datestring = '201501'
getCitiBikeCSV(datestring)

('Downloading', '201501')
file in place, you can continue


In [13]:
data = pd.read_csv('PUIdata/' + datestring + "-citibike-tripdata.csv")

In [14]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1346,1/1/2015 0:01,1/1/2015 0:24,455,1 Ave & E 44 St,40.750020,-73.969053,265,Stanton St & Chrystie St,40.722293,-73.991475,18660,Subscriber,1960.0,2
1,363,1/1/2015 0:02,1/1/2015 0:08,434,9 Ave & W 18 St,40.743174,-74.003664,482,W 15 St & 7 Ave,40.739355,-73.999318,16085,Subscriber,1963.0,1
2,346,1/1/2015 0:04,1/1/2015 0:10,491,E 24 St & Park Ave S,40.740964,-73.986022,505,6 Ave & W 33 St,40.749013,-73.988484,20845,Subscriber,1974.0,1
3,182,1/1/2015 0:04,1/1/2015 0:07,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,19610,Subscriber,1969.0,1
4,969,1/1/2015 0:05,1/1/2015 0:21,474,5 Ave & E 29 St,40.745168,-73.986831,432,E 7 St & Avenue A,40.726218,-73.983799,20197,Subscriber,1977.0,1


In [15]:
data['age'] = 2016 - data['birth year']

In [16]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,age
0,1346,1/1/2015 0:01,1/1/2015 0:24,455,1 Ave & E 44 St,40.750020,-73.969053,265,Stanton St & Chrystie St,40.722293,-73.991475,18660,Subscriber,1960.0,2,56.0
1,363,1/1/2015 0:02,1/1/2015 0:08,434,9 Ave & W 18 St,40.743174,-74.003664,482,W 15 St & 7 Ave,40.739355,-73.999318,16085,Subscriber,1963.0,1,53.0
2,346,1/1/2015 0:04,1/1/2015 0:10,491,E 24 St & Park Ave S,40.740964,-73.986022,505,6 Ave & W 33 St,40.749013,-73.988484,20845,Subscriber,1974.0,1,42.0
3,182,1/1/2015 0:04,1/1/2015 0:07,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,19610,Subscriber,1969.0,1,47.0
4,969,1/1/2015 0:05,1/1/2015 0:21,474,5 Ave & E 29 St,40.745168,-73.986831,432,E 7 St & Avenue A,40.726218,-73.983799,20197,Subscriber,1977.0,1,39.0


In [31]:
data.describe()

/Users/christianrosado/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender,age
count,285552.000000,285552.000000,285552.000000,285552.000000,285552.000000,285552.000000,285552.000000,285552.000000,279924.000000,285552.000000,279924.000000
mean,654.325583,443.535006,40.735804,-73.990177,440.337557,40.735607,-73.990398,18147.291481,1974.951419,1.162485,41.048581
std,900.775928,332.580265,0.018773,0.011681,326.010259,0.018718,0.011742,2065.313794,11.574637,0.419490,11.574637
min,60.000000,72.000000,40.680342,-74.017134,72.000000,40.680342,-74.017134,14529.000000,1899.000000,0.000000,17.000000
25%,334.000000,308.000000,40.723627,-73.998102,307.000000,40.723180,-73.998522,16355.000000,NaN,1.000000,NaN
50%,504.000000,417.000000,40.738274,-73.990093,417.000000,40.737815,-73.990617,18148.000000,NaN,1.000000,NaN
75%,772.000000,492.000000,40.750450,-73.981854,492.000000,40.750200,-73.981854,19903.000000,NaN,1.000000,NaN
max,43023.000000,3002.000000,40.771522,-73.950048,3002.000000,40.771522,-73.950048,21690.000000,1999.000000,2.000000,117.000000


In [42]:
age_data = data['age'][data['usertype']=='Subscriber']
type(age_data)

pandas.core.series.Series

# KS Test 1

H0: The age distribution <span style="color: #66A366;"><b>is</b></span> drawn from a <span style="color: #000099;"><b>Gaussian</b></span> distribution <br >
H1: The age distribution <span style="color: #B20000;"><b>is not</b></span> drawn from a <span style="color: #000099;"><b>Gaussian</b></span> distribution <br >
<span style="color: #B20000;">alpha = 0.05</span>

In [49]:
stats.kstest(age_data, 'norm', args=(age_data.mean(), age_data.std()))

KstestResult(statistic=0.087552112722919739, pvalue=0.0)

After running our KS test, we are able to Reject our null due to a very low pvalue. <br >
<span style="color: #FF69B4;">statistic=0.087552112722919739, pvalue=0.0</span>

# KS Test 2

H0: The age distribution <span style="color: #66A366;"><b>is</b></span> drawn from a <span style="color: #000099;"><b>Chi2</b></span> distribution<br >
H1: The age distribution <span style="color: #B20000;"><b>is not</b></span> drawn from a <span style="color: #000099;"><b>Chi2</b></span> distribution<br >
<span style="color: #B20000;">alpha = 0.05</span>

In [50]:
stats.kstest(age_data, 'chi2', args=(age_data.mean(), age_data.std()))

KstestResult(statistic=0.47193760369578153, pvalue=0.0)

After running our KS test, we are able to Reject our null due to a very low pvalue. <br >
<span style="color: #FF69B4;">statistic=0.087552112722919739, pvalue=0.0</span>

# AD Test 1

H0: The age distribution <span style="color: #66A366;"><b>is</b></span> drawn from a <span style="color: #000099;"><b>Gaussian</b></span> distribution<br >
H1: The age distribution <span style="color: #B20000;"><b>is not</b></span> drawn from a <span style="color: #000099;"><b>Gaussian</b></span> distribution<br >
<span style="color: #B20000;">alpha = 0.05</span>

In [45]:
stats.anderson(age_data)

AndersonResult(statistic=2732.9753468575072, critical_values=array([ 0.576,  0.656,  0.787,  0.918,  1.092]), significance_level=array([ 15. ,  10. ,   5. ,   2.5,   1. ]))

After running our AD test, we are able to Reject our null due to a very high critical value. <br >
<span style="color: #FF69B4;">critical value=0.787</span>

# AD Test 2

H0: The age distribution <span style="color: #66A366;"><b>is</b></span> drawn from a <span style="color: #000099;"><b>Exponential</b></span> distribution<br >
H1: The age distribution <span style="color: #B20000;"><b>is not</b></span> drawn from a <span style="color: #000099;"><b>Exponential</b></span> distribution<br >
<span style="color: #B20000;">alpha = 0.05</span>

In [51]:
stats.anderson(age_data, 'expon')

AndersonResult(statistic=67390.930461328069, critical_values=array([ 0.922,  1.078,  1.341,  1.606,  1.957]), significance_level=array([ 15. ,  10. ,   5. ,   2.5,   1. ]))

After running our AD test, we are able to Reject our null due to a very high critical value. <br >
<span style="color: #FF69B4;">critical value=1.341</span>